# Exploring the Decay Position Network

This is repeating a little bit of the work that Rachel did - but going straight to a training. This notebook will use keras.

## Config

In [1]:
func_adl_endpoint = 'http://localhost:8000'
datasets_for_training_datafile = "../data/datasets.csv"

# Constants derived in previous notebook. Need to be added to a python file of config constants.
lxyz_eta_division = 1.3
too_far_dist_lz = 7500
too_far_dist_lxy = 4400
too_short_dist_lxy=1300
too_short_dist_lz=3500

# How many events per training sample shall we train on?
training_events_per_sample = 4000
epochs_to_train = 250

# Columns to train on. This is partly gotten by looking at the `Input Variables` worksheet to remove blanks.
what_to_train_on = ['EMM_BL0', 'EMM_BL1', 'EMM_BL2',
       'EMM_BL3', 'EMM_EL0', 'EMM_EL1', 'EMM_EL2', 'EMM_EL3', 'EH_EL0',
       'EH_EL1', 'EH_EL2', 'EH_CBL0', 'EH_CBL1', 'EH_CVL2',
       'EH_TGL0', 'EH_TGL1', 'EH_TGL2', 'EH_EBL0', 'EH_EBL1', 'EH_EBL2', 'JetPt', 'JetEta']
# With an eta cut of 1.3, then EH_EL3 is also all zeros.
#  'FC_L0', 'FC_L1', 'FC_L2' - these seem to be all zeros as seen before.

## Python setup

In [2]:
# Designed not to be modified
import sys
sys.path.append("../")
from adl_func_client.event_dataset import EventDataset
from adl_func_client.use_exe_func_adl_server import use_exe_func_adl_server
from calratio_perjet_training.fetch_training_data import fetch_perjet_data
import glob
import numpy as np
import asyncio

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib import rcParams
plt.rc('font', size=14)
import seaborn as sns

import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

from keras.models import Sequential
from keras.layers import Dense, Dropout
import sklearn.metrics

Using TensorFlow backend.


## Load datasets

In [3]:
datasets = pd.read_csv(datasets_for_training_datafile)

In [4]:
async def fetch_data_async(info):
    return [info, f'{info.mH}_{info.mS}_{info.Lifetime}_{info.MCCampaign}', await fetch_perjet_data(EventDataset(f'localds://{info.RucioDSName}'), f'{info.mH}_{info.mS}_{info.Lifetime}_{info.MCCampaign}')]
all_datasets_future = [fetch_data_async(info) for index, info in datasets.iterrows()]
datasets_for_training = await asyncio.gather(*all_datasets_future)

We need to split into training and testing sample sizes. Unfortunately, we have to do some calculations to understand if something is good signal to train on, or not. So we add a few columns here to all the datasets.

In [ ]:
def add_useful_columns(ds):
    ds['Lxy'] = np.sqrt(ds.Lx*ds.Lx + ds.Ly*ds.Ly)

    ds['IsOutlier'] = False
    ds['IsOutlier'] |= ds.Lxy[ds.IsLLP & (np.abs(ds.JetEta) < lxyz_eta_division)] > too_far_dist_lxy
    ds['IsOutlier'] |= ds.Lz[ds.IsLLP & (np.abs(ds.JetEta) >= lxyz_eta_division)] > too_far_dist_lz

    ds['IsInlier'] = False
    ds['IsInlier'] |= ds.Lxy[ds.IsLLP & (np.abs(ds.JetEta) < lxyz_eta_division)] < too_short_dist_lxy
    ds['IsInlier'] |= ds.Lz[ds.IsLLP & (np.abs(ds.JetEta) >= lxyz_eta_division)] < too_short_dist_lz

    ds['JetIsCentral'] = np.abs(ds.JetEta) < lxyz_eta_division

    ds["Signal"] = ds.IsLLP & (ds.JetPt > 40) & (np.abs(ds.JetEta) < 2.4) & (ds.IsOutlier == False) & (ds.IsInlier == False) & (ds.JetIsCentral == True)

# We can limit how many datasets we combine to make life a little easier for testing with this line
what_to_combine = datasets_for_training #[20:35]
for d in what_to_combine:
    add_useful_columns(d[2])

In [ ]:
all_training_jets = pd.DataFrame(pd.concat([d[2][d[2].Signal][:training_events_per_sample] for d in what_to_combine], keys=[(d[0].mH, d[0].mS, d[0].Lifetime, f'{d[0].mH}/{d[0].mS}', d[0].MCCampaign) for d in what_to_combine], names=['mH', 'mS', 'Lifetime', 'mH_mS', 'MC']).to_records())
all_testing_jets = pd.DataFrame(pd.concat([d[2][d[2].Signal][training_events_per_sample+1:] for d in what_to_combine], keys=[(d[0].mH, d[0].mS, d[0].Lifetime, f'{d[0].mH}/{d[0].mS}', d[0].MCCampaign) for d in what_to_combine], names=['mH', 'mS', 'Lifetime', 'mH_mS', 'MC']).to_records())

In [ ]:
print (f'Number of training jets: {len(all_training_jets)}')
print (f'Number of testing jets: {len(all_testing_jets)}')

And quick reference for the columns we have in the data

In [ ]:
all_training_jets.columns

## Normalization

For this training best to center thinngs around the average before doing the training (or inference). Create some tools to do that.

In [ ]:
def calc_normalization (p):
    return (p.mean(), p.std())

(input_mean, input_std) = calc_normalization(all_training_jets.filter(items=what_to_train_on))
(output_mean, output_std) = calc_normalization(all_training_jets.filter(items=['Lxy']))

def norm_inputs(p):
    return (p - input_mean) / input_std

def norm_outputs(p):
    return (p - output_mean[0]) / output_std[0]

def unnorm_outputs(p):
    return (p * output_std[0]) + output_mean[0]

In [ ]:
y_train = norm_outputs(all_training_jets.Lxy)
x_train = norm_inputs(all_training_jets.filter(items=what_to_train_on))

In [ ]:
x_train

## Build the Model

In [ ]:
%%capture --no-stdout --no-display

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=len(x_train.columns)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
# THis is the number of outputs - so could be 2 if we wanted to train both lxy and lz
# Activation might be softmax if we had more than one thing as we would would want it to some to some number.
# But since this is regression, we do not.
model.add(Dense(1))

#categorical_crossentropy
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, batch_size=32, epochs=epochs_to_train, validation_split=0.25, shuffle=True, verbose=0);

## Predict

This is a little tricky in the sense we want to run the prediction for the whole table. We split out the testing stuff above. So normalize it and we are ready to go!
Note we have to un-normalize things before we get to plotting and comparing!

In [ ]:
x_test = norm_inputs(all_testing_jets.filter(items=what_to_train_on))

In [ ]:
y_predict = model.predict(x_test)

In [ ]:
all_testing_jets['p_Lxy'] = np.array(unnorm_outputs(y_predict[:,0]))

## Comparisons

Lets look at how well the prediction does vs various things

In [ ]:
all_testing_jets

In [ ]:
plt.scatter(x=all_testing_jets.Lxy/1000.0, y=all_testing_jets.p_Lxy/1000.0)
plt.xlabel('Actual value of $L_{xy}$')
plt.ylabel('Predicted value of $L_{xy}$')
plt.show()

In [ ]:
mass_comparison = sns.relplot(x='Lxy', y='p_Lxy', kind='scatter', data=all_testing_jets, row='mH', col='mS', height=6, aspect=1)
mass_comparison.set(ylim=(0.0, 5000.0))
mass_comparison.set(xlim=(0.0, 5000.0))

In [ ]:
what_to_train_on

## The Calorimeter Edge Effect

Looking at these plots there are many of them that have a seeming edge - a group of jets that have $L_{xy}$ that run along the calorimeter, but are all predicted to be at the face. My guess is this is connected with pileup somehow. So this is just an irreducable background.

First, lets isolate them.

In [ ]:
all_testing_jets['EdgeBadReco'] = (all_testing_jets.p_Lxy < 1750) & (all_testing_jets.Lxy > 2500)

In [ ]:
mass_comparison = sns.relplot(x='Lxy', y='p_Lxy', kind='scatter', hue='EdgeBadReco', data=all_testing_jets, height=6, aspect=1)
mass_comparison.set(ylim=(0.0, 5000.0))
mass_comparison.set(xlim=(0.0, 5000.0))

In [ ]:
mass_comparison = sns.relplot(x='Lxy', y='p_Lxy', kind='scatter', hue='EdgeBadReco', data=all_testing_jets, row='mH', col='mS', height=6, aspect=1)
mass_comparison.set(ylim=(0.0, 5000.0))
mass_comparison.set(xlim=(0.0, 5000.0))

### Kinematics?

Get an idea behind simple kinematics.

In [ ]:
#sns.distplot(x='JetPt', hue='EdgeBadReco', data=all_testing_jets)
g = sns.FacetGrid(all_testing_jets[all_testing_jets.JetPt < 1000], col="mH", hue='EdgeBadReco', height=5)
g.map(sns.distplot, 'JetPt', norm_hist=False, kde=False)
g.set(yscale='log')
g.add_legend()

In [ ]:
#sns.distplot(x='JetPt', hue='EdgeBadReco', data=all_testing_jets)
g = sns.FacetGrid(all_testing_jets[all_testing_jets.JetPt < 1000], col="mH", hue='EdgeBadReco', height=5)
g.map(sns.distplot, 'JetEta', norm_hist=False, kde=False)
g.set(yscale='log')
g.add_legend()

In [ ]:
#sns.distplot(x='JetPt', hue='EdgeBadReco', data=all_testing_jets)
g = sns.FacetGrid(all_testing_jets[all_testing_jets.JetPt < 1000], col="mH", hue='EdgeBadReco', height=5)
g.map(sns.distplot, 'JetPhi', norm_hist=False, kde=False)
g.set(yscale='log')
g.add_legend()

In [ ]:
#sns.distplot(x='JetPt', hue='EdgeBadReco', data=all_testing_jets)
g = sns.FacetGrid(all_testing_jets[all_testing_jets.JetPt < 1000], col="mH", hue='EdgeBadReco', height=5)
g.map(plt.scatter, 'JetEta', 'JetPhi')
g.add_legend()

Lets look at the inner layer of the EM. We know from the input variables plot that it is not really responding for an $L_{xy} > 1750$ mm. So that means that we should see little energy there. If there is another jet - like a pileup jet, then we should see an energy deposit there.

In [ ]:
#sns.distplot(x='JetPt', hue='EdgeBadReco', data=all_testing_jets)
g = sns.FacetGrid(all_testing_jets[(all_testing_jets.JetPt < 1000) & (all_testing_jets.Lxy > 1750)], col="mH", hue='EdgeBadReco', height=5)
g.map(sns.distplot, 'EMM_BL0', norm_hist=False, kde=False)
g.set(yscale='log')
g.add_legend()

I admit to being a bit suprised. This is inconclusive:

- On one hand, it the well reconstructed often has lower energy than this guy.
- But that isn't always the case!

We can definately conclude that EMM_BL0 is higher for these funny jets. But it isn't black & white.